In [1]:
import os
import sys
import time
#sys.path.append('../..')

from datasets.get_datasets import *
from boostedrevision import *
from boostedtransfer import *
from boostsrl import boostsrl
from mapping import *
import numpy as np
import random
import json

In [16]:
bk = {
      'imdb': ['workedunder(+person,+person).',
              'workedunder(+person,-person).',
              'workedunder(-person,+person).',
              'female(+person).',
              'actor(+person).',
              'director(+person).',
              'movie(+movie,+person).',
              'movie(+movie,-person).',
              'movie(-movie,+person).',
              'genre(+person,+genre).'],
      'uwcse': ['professor(+person).',
        'student(+person).',
        'advisedby(+person,+person).',
        'advisedby(+person,-person).',
        'advisedby(-person,+person).',
        'tempadvisedby(+person,+person).',
        'tempadvisedby(+person,-person).',
        'tempadvisedby(-person,+person).',
        #'ta(+course,+person,+quarter).',
        #'ta(-course,+person,+quarter).',
        #'ta(+course,-person,+quarter).',
        #'ta(+course,+person,-quarter).',
        #'ta(-course,+person,-quarter).',
        #'ta(+course,-person,-quarter).',
        'hasposition(+person,+faculty).',
        'hasposition(+person,-faculty).',
        'hasposition(-person,+faculty).',
        'publication(+title,+person).',
        'publication(+title,-person).',
        'publication(-title,+person).',
        'inphase(+person,+prequals).',
        'inphase(+person,-prequals).',
        'inphase(-person,+prequals).',
        'courselevel(+course,+level).',
        'courselevel(+course,-level).',
        'courselevel(-course,+level).',
        'yearsinprogram(+person,+year).',
        'yearsinprogram(-person,+year).',
        'yearsinprogram(+person,-year).',
        'projectmember(+project,+person).',
        'projectmember(+project,-person).',
        'projectmember(-project,+person).'],}

In [4]:
experiments = [
            {'source':'imdb', 'target':'uwcse', 'predicate':'workedunder'}]

In [5]:
        source = 'imdb'
        target = 'uwcse'
        predicate = 'workedunder'

In [8]:
src_total_data = datasets.load(source, bk[source])
src_data = datasets.load(source, bk[source], target=predicate)

# Group and shuffle
src_facts = datasets.group_folds(src_data[0])
src_pos = datasets.group_folds(src_data[1])
src_neg = datasets.group_folds(src_data[2])

background = boostsrl.modes(bk[source], [predicate], useStdLogicVariables=False, maxTreeDepth=8, nodeSize=3, numOfClauses=8)

[model, total_revision_time, source_structured, will, variances] = learn_model(background, boostsrl, predicate, src_pos, src_neg, src_facts, refine=None, trees=10, verbose=True)

WILL Produced-Tree #1
% FOR workedunder(A, B):
%   if ( director(B), movie(C, A), movie(C, B) )
%   then if ( female(A) )
%   | then return 0.8581489350995096;  // std dev = 1,07e-06, 170,000 (wgt'ed) examples reached here.  /* #pos=170 */
%   | else return 0.8396304165809906;  // std dev = 0,135, 216,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=212 */
%   else return -0.14185106490048766;  // std dev = 1,88e-07, 378,000 (wgt'ed) examples reached here.  /* #neg=378 */
WILL Produced-Tree #2
% FOR workedunder(A, B):
%   if ( director(B), movie(C, A), movie(C, B) )
%   then if ( female(A) )
%   | then return 0.7194734122109565;  // std dev = 0,000, 170,000 (wgt'ed) examples reached here.  /* #pos=170 */
%   | else return 0.7046772772613799;  // std dev = 0,135, 216,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=212 */
%   else return -0.12544463852839208;  // std dev = 0,000, 378,000 (wgt'ed) examples reached here.  /* #neg=378 */
WILL Produced-Tree #3
% FOR workedunder(A, B):

In [14]:
preds = mapping.get_preds(source_structured, bk[source])
print('\n')
print('Predicates from source: %s \n' % preds)
#print('Source structured tree: %s \n' % source_structured)



Predicates from source: ['workedunder(person,person).', 'director(person).', 'movie(movie,person).', 'female(person).'] 



In [17]:


# Load total target dataset
tar_total_data = datasets.load(target, bk[target])

[tar_train_pos, tar_test_pos] = datasets.get_kfold_small(0, tar_total_data[0])

# transfer
mapping_rules, mapping_results = mapping.get_best(preds, bk[target], datasets.group_folds(src_total_data[0]), tar_train_pos)
transferred_structured = transfer(source_structured, mapping_rules)
new_target = get_transferred_target(transferred_structured)

print('Best mapping found: %s \n' % mapping_rules)
#print('Tranferred structured tree: %s \n' % transferred_structured)
print('Transferred target predicate: %s \n' % new_target)

Best mapping found: ['workedunder(A,B) -> advisedby(A,B)', 'director(A) -> professor(A)', 'female(A) -> student(A)', 'movie(A,B) -> publication(A,B)'] 

Transferred target predicate: advisedby 



In [21]:
# Load new predicate target dataset
tar_data = datasets.load(target, bk[target], target=new_target)

i = 0
# Group and shuffle
[tar_train_facts, tar_test_facts] =  datasets.get_kfold_small(i, tar_data[0])
[tar_train_pos, tar_test_pos] =  datasets.get_kfold_small(i, tar_data[1])
[tar_train_neg, tar_test_neg] =  datasets.get_kfold_small(i, tar_data[2])

validation_pos = tar_test_pos
validation_neg = tar_test_neg

In [23]:
# transfer and parameter learning
background = boostsrl.modes(bk[target], [new_target], useStdLogicVariables=False, maxTreeDepth=8, nodeSize=3, numOfClauses=8)
[model, total_revision_time, structured, will, variances] = learn_model(background, boostsrl, predicate, tar_train_pos, tar_train_neg, tar_train_facts, refine=get_boosted_refine_file(transferred_structured), trees=10, verbose=True)
#[model, total_revision_time, inference_time, t_results, structured, pl_t_results] = theory_revision(background, boostsrl, target, tar_train_pos, tar_train_neg, tar_train_facts, validation_pos, validation_neg, tar_test_pos, tar_test_neg, tar_test_facts, transferred_structured, trees=10, max_revision_iterations=10, testAfterPL=True, verbose=True)

WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( professor(B), publication(C, A), publication(C, B) )
%   then if ( student(A) )
%   | then return 0.8581489350995123;  // std dev = 0,000, 14,000 (wgt'ed) examples reached here.  /* #pos=14 */
%   | else return -0.14185106490048777;  // std dev = 0,000, 1,000 (wgt'ed) examples reached here.  /* #neg=1 */
%   else return 0.23996711691769412;  // std dev = 3,603, 55,000 (wgt'ed) examples reached here.  /* #neg=34 #pos=21 */
WILL Produced-Tree #2
% FOR advisedby(A, B):
%   if ( professor(B), publication(C, A), publication(C, B) )
%   then if ( student(A) )
%   | then return 0.7194734122109542;  // std dev = 5,16e-08, 14,000 (wgt'ed) examples reached here.  /* #pos=14 */
%   | else return -0.12544463852839138;  // std dev = 0,000, 1,000 (wgt'ed) examples reached here.  /* #neg=1 */
%   else return 0.20817625325674946;  // std dev = 3,603, 55,000 (wgt'ed) examples reached here.  /* #neg=34 #pos=21 */
WILL Produced-Tree #3
% FOR advisedby

In [24]:
candidate = get_boosted_candidate(structured, variances)

In [25]:
candidate

['0;;advisedby(A, B) :- professor(B), publication(C, A), publication(C, B).;true;true',
 '0;true;student(A).;true;true',
 '1;;advisedby(A, B) :- professor(B), publication(C, A), publication(C, B).;true;true',
 '1;true;student(A).;true;true',
 '2;;advisedby(A, B) :- professor(B), publication(C, A), publication(C, B).;true;true',
 '2;true,false;professor(A).;true;true',
 '2;true;student(A).;true;true',
 '3;;advisedby(A, B) :- professor(B), publication(C, A), publication(C, B).;true;true',
 '3;true;student(A).;true;true',
 '4;;advisedby(A, B) :- professor(B), publication(C, A), publication(C, B).;true;true',
 '4;true;student(A).;true;true',
 '5;;advisedby(A, B) :- professor(B), publication(C, A), publication(C, B).;true;true',
 '5;true;student(A).;true;true',
 '6;;advisedby(A, B) :- professor(B), publication(C, A), publication(C, B).;true;true',
 '6;true;professor(A).;true;true',
 '7;;advisedby(A, B) :- professor(B), publication(C, A), publication(C, B).;true;true',
 '7;true;student(A).;t

In [26]:
# transfer and parameter learning
background = boostsrl.modes(bk[target], [new_target], useStdLogicVariables=False, maxTreeDepth=8, nodeSize=3, numOfClauses=8)
[model, total_revision_time, structured, will, variances] = learn_model(background, boostsrl, predicate, tar_train_pos, tar_train_neg, tar_train_facts, refine=candidate, trees=10, verbose=True)
#[model, total_revision_time, inference_time, t_results, structured, pl_t_results] = theory_revision(background, boostsrl, target, tar_train_pos, tar_train_neg, tar_train_facts, validation_pos, validation_neg, tar_test_pos, tar_test_neg, tar_test_facts, transferred_structured, trees=10, max_revision_iterations=10, testAfterPL=True, verbose=True)

WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( professor(B), publication(C, A), publication(C, B) )
%   then if ( student(A) )
%   | then return 0.8581489350995123;  // std dev = 0,000, 14,000 (wgt'ed) examples reached here.  /* #pos=14 */
%   | else return -0.14185106490048777;  // std dev = 0,000, 1,000 (wgt'ed) examples reached here.  /* #neg=1 */
%   else if ( professor(B), student(A) )
%   | then if ( tempadvisedby(D, B), publication(E, D) )
%   | | then return 0.0581489350995122;  // std dev = 0,894, 5,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=1 */
%   | | else if ( tempadvisedby(F, B) )
%   | | | then if ( projectmember(G, B), publication(H, A) )
%   | | | | then return 0.8581489350995121;  // std dev = 2,11e-08, 3,000 (wgt'ed) examples reached here.  /* #pos=3 */
%   | | | | else if ( publication(I, B) )
%   | | | | | then return 0.4295775065280837;  // std dev = 0,495, 7,000 (wgt'ed) examples reached here.  /* #neg=3 #pos=4 */
%   | | | | | else return 0.35814